# Project 3: Airbnb
**This is the third of three mandatory projects to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The project is to be done in groups of at most 3 students**
- **Each group has to hand in _one_ Jupyter notebook (this notebook) with their solution**
- **The hand-in of the notebook is due 2019-12-05, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python/PySpark**
- **For each question you may use as many cells for your solution as you like**
- **You should not remove the problem statements**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**
- **You are not expected to use machine learning to solve any of the exercises**

# Introduction
[Airbnb](http://airbnb.com) is an online marketplace for arranging or offering lodgings. In this project you will use Spark to analyze data obtained from the Airbnb website. The purpose of the analysis is to extract information about trends and patterns from the data.

The project has two parts.

### Part 1: Loading, describing and preparing the data
There's quite a lot of data. Make sure that you can load and correctly parse the data, and that you understand what the dataset contains. You should also prepare the data for the analysis in part two. This means cleaning it and staging it so that subsequent queries are fast.

### Par 2: Analysis
In this part your goal is to learn about trends and usage patterns from the data. You should give solutions to the tasks defined in this notebook, and you should use Spark to do the data processing. You may use other libraries like for instance Pandas and matplotlib for visualisation.

## Guidelines
- Processing data should be done using Spark. Once data has been reduced to aggregate form, you may use collect to extract it into Python for visualisation.
- Your solutions will be evaluated by correctness, code quality and interpretability of the output. This means that you have to write clean and efficient Spark code that will generate sensible execution plans, and that the tables and visualisations that you produce are meaningful and easy to read.
- You may add more cells for your solutions, but you should not modify the notebook otherwise.

### Create Spark session and define imports

In [1]:
from pyspark.sql import *
from pyspark.sql import functions as f
from pyspark.sql.types import *
import os

os.environ["PYSPARK_PYTHON"] = "/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7"

spark = SparkSession.builder.appName("Project3").getOrCreate()

# Part 1: Loading, describing and preparing the data
The data comes in two files. Start by downloading the files and putting them in your `data/` folder.

- [Listings](https://files.dtu.dk/u/siPzAasj8w2gI_ME/listings.csv?l) (5 GB)
- [Reviews](https://files.dtu.dk/u/k3oaPYp6GjKBeho4/reviews.csv?l) (9.5 GB)

### Load the data
The data has multiline rows (rows that span multiple lines in the file). To correctly parse these you should use the `multiline` option and set the `escape` character to be `"`.

In [2]:
df = spark.read.option('header', True).option('inferSchema', True).option('escape', '"').option('multiLine', True).csv('/Users/isma/Desktop/data-science/02807-comp-tools/docker/work/data/listings.csv')
df_reviews = spark.read.option("multiLine", True).option('header', True).option("escape", "\"").csv('/Users/isma/Desktop/data-science/02807-comp-tools/docker/work/data/reviews.csv')

### Describe the data
List the features (schema) and sizes of the datasets.

The data consists of:
- id (integer?)
- listing_url (string)
- scrape_id (integer?)
- last_scraped (date?)
- name (string)
- summary (string)
- space (integer?)
- description (string)
- experiences_offered (integer?)
- neighborhood_overview
- notes (string)
- transit
- access
- interaction
- house_rules (string)
- thumbnail_url (string)
- medium_url (string)
- picture_url (string)
- xl_picture_url (string)
- host_id (integer?)
- host_url (string)
- host_name (string)
- host_since (date?)
- host_location (city?)
- host_about (string)
- host_response_time (integer?)
- host_response_rate (number %)
- host_acceptance_rate (number %)
- host_is_superhost (boolean)
- host_thumbnail_url (string)
- host_picture_url (string)
- host_neighbourhood (string)
- host_listings_count (integer?)
- host_total_listings_count (integer?)
- host_verifications (integer?)
- host_has_profile_pic (boolean?)
- host_identity_verified (boolean?)
- street (string)
- neighbourhood (string)
- neighbourhood_cleansed
- neighbourhood_group_cleansed
- city (string)
- state (string)
- zipcode (integer?)
- market (string)
- smart_location
- country_code (string) 2 digits
- country (string)
- latitude (integer?)
- longitude (integer?)
- is_location_exact (boolean?)
- property_type ???
- room_type ???
- accommodates (integer?)
- bathrooms (integer?)
- bedrooms (integer?)
- beds (integer?)
- bed_type ???
- amenities (integer?)
- square_feet (integer?)
- price (integer?)
- weekly_price (integer?)
- monthly_price (integer?)
- security_deposit (integer?)
- cleaning_fee (integer?)
- guests_included ???
- extra_people (integer?)
- minimum_nights (integer?)
- maximum_nights (integer?)
- minimum_minimum_nights (integer?)
- maximum_minimum_nights (integer?)
- minimum_maximum_nights (integer?)
- maximum_maximum_nights (integer?)
- minimum_nights_avg_ntm (integer?)
- maximum_nights_avg_ntm (integer?)
- calendar_updated ???
- has_availability (boolean?)
- availability_30 ???
- availability_60 ???
- availability_90 ???
- availability_365 ???
- calendar_last_scraped (date?)
- number_of_reviews (integer?)
- number_of_reviews_ltm (integer?)
- first_review ???
- last_review ???
- review_scores_rating  (float?)
- review_scores_accuracy  (float?)
- review_scores_cleanliness  (float?)
- review_scores_checkin  (float?)
- review_scores_communication  (float?)
- review_scores_location  (float?)
- review_scores_value  (float?)
- requires_license  (float?)
- license ???
- jurisdiction_names (string)
- instant_bookable (boolean?)
- is_business_travel_ready (boolean?)
- cancellation_policy ???
- require_guest_profile_picture (boolean?)
- require_guest_phone_verification (boolean?)
- calculated_host_listings_count (integer?)
- calculated_host_listings_count_entire_homes (integer?)
- calculated_host_listings_count_private_rooms (integer?)
- calculated_host_listings_count_shared_rooms (integer?)
- reviews_per_month (integer?)

Print the schema of the dataset

In [22]:
df_reviews.show(5)
df.printSchema()

+----------+---------+----------+-----------+-------------+--------------------+
|listing_id|       id|      date|reviewer_id|reviewer_name|            comments|
+----------+---------+----------+-----------+-------------+--------------------+
|    145320|156423122|2017-05-30|  123386382|        Erwin|Prima plek om Sto...|
|    145320|170211906|2017-07-15|  123091743|         Anne|Cosy and clean fl...|
|    145320|172169175|2017-07-20|      78004|     Patricia|The host canceled...|
|    145320|176647581|2017-07-31|  103178743|    Charlotte|Kim's place was o...|
|    145320|185676021|2017-08-22|    4023961|    Alexander|great spacious ap...|
+----------+---------+----------+-----------+-------------+--------------------+
only showing top 5 rows

root
 |-- id: string (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- scrape_id: string (nullable = true)
 |-- last_scraped: string (nullable = true)
 |-- name: string (nullable = true)
 |-- summary: string (nullable = true)
 |--

Number of rows:

In [4]:
print("Reviews file rows: " + str(df_reviews.count()) + ". Reviews file columns: " + str(len(df_reviews.columns)))
print("Listings file rows: " + str(df.count()) + ". Reviews file columns: " + str(len(df.columns)))

Reviews file rows: 32297300. Reviews file columns: 6
Listings file rows: 1330480. Reviews file columns: 106


### Prepare the data for analysis
You should prepare two dataframes to be used in the analysis part of the project. You should not be concerned with cleaning the data. There's a lot of it, so it will be sufficient to drop rows that have bad values. You may want to go back and refine this step at a later point when doing the analysis.

You may also want to consider if you can stage your data so that subsequent processing is more efficient (this is not strictly necessary for Spark to run, but you may be able to decrease the time you sit around waiting for Spark to finish things)

In [20]:
# drop all rows with null values
df = df.drop('all')

# TODO: drop all null prices and rating values (I guess drop the whole row)
listings_reduced = \
    df.select( \
        f.col("id"), \
        f.col("neighbourhood"), \
        f.col("city"), \
        f.col("country"),  \
        f.col("property_type"), \
        f.col("price"), \
        f.col("reviews_per_month"), \
        f.col("review_scores_rating")) \
    .filter( \
        f.col('neighbourhood').isNotNull() & \
        f.col('city').isNotNull() & \
        f.col('country').isNotNull() & \
        f.col('property_type').isNotNull() & \
        f.col('price').isNotNull() & \
        f.col('review_scores_rating').isNotNull())

listings_reduced.show(20)


+------+-------------+-----------+-------+-----------------+---------+-----------------+--------------------+
|    id|neighbourhood|       city|country|    property_type|    price|reviews_per_month|review_scores_rating|
+------+-------------+-----------+-------+-----------------+---------+-----------------+--------------------+
|145320|    Södermalm|  Stockholm| Sweden|        Apartment|$1,272.00|             2.54|                  97|
|155685|    Södermalm|  Stockholm| Sweden|        Apartment|$3,209.00|             0.22|                  94|
|164448|    Södermalm|  Stockholm| Sweden|        Apartment|  $597.00|             3.13|                  97|
|170651|    Södermalm|  Stockholm| Sweden|        Apartment|  $734.00|             0.33|                  93|
|206221|    Södermalm|  Stockholm| Sweden|Bed and breakfast|  $665.00|             0.82|                  98|
|220851|  Kungsholmen|  Stockholm| Sweden|        Apartment|  $597.00|             0.46|                  93|
|242188|  

In [ ]:
df.groupBy('country').agg(f.count('id')).show()

# Part 2: Analysis
Use Spark and your favorite tool for data visualization to solve the following tasks.

## The basics
Compute and show a dataframe with the number of listings and neighbourhoods per city.

In [ ]:
basic_df = df.select('city', 'neighbourhood', 'host_listings_count', 'property_type', 'price')
basic_query = basic_df.groupBy('city').agg(f.count('neighbourhood').alias('number_neighbourhoods'), f.count('host_listings_count').alias('number_listings'))
# .agg(f.count('host_listings_count').alias('number_listings')
basic_query.show()


Based on the table above, you should choose a city that you want to continue your analysis for. The city should have mulitple neighbourhoods with listings in them.

Compute and visualize the number of listings of different property types per neighbourhood in your city.

In [ ]:
basic_second_df = basic_df.filter(f.col('city') == 'Madrid').select('neighbourhood', 'host_listings_count', 'property_type')
basic_second_df = basic_second_df.dropna()
basic_second_query = basic_second_df.groupBy('neighbourhood','property_type').agg(f.count('host_listings_count')).orderBy('neighbourhood').show()




## Prices
Compute the minimum, maximum and average listing price in your city. 

In [ ]:
madrid_prices = basic_df.filter(f.col('city') == 'Madrid').select('price').drop('all')
madrid_prices.show()


I want to do a function that takes away the "$" symbol and the "," from the prices, so we can then convert them to float type and get the average price

In [ ]:
def extract_price(price):
    price = price.lstrip('$').replace(',', '')
    return price

In [ ]:
extract_price_udf = f.udf(extract_price)

This is not working, don't know why. It's the same thing Patrick did in class on "Introduction to Spark"

In [ ]:
#import org.apache.spark.sql.types.FloatType

madrid_prices2 = madrid_prices.withColumn('price_new', extract_price_udf(f.col('price'))).select('price_new')
madrid_prices_parsed = madrid_prices2.withColumn("price", f.col("price_new").cast("float")).drop('price_new')
madrid_prices_parsed.show(5)
madrid_prices_parsed.printSchema()


- Average does not work since price is still string type
- Min price is wrong since we need to take out the commas 

In [ ]:
# Min, max, and average price
madrid_prices_parsed.select(f.min('price'), f.max('price'), f.avg('price')).show()
row1 = madrid_prices_parsed.agg({"price": "max"}).collect()[0]


Compute and visualize the distribution of listing prices in your city.

In [ ]:
# Don't know what "distribution of listing prices" means honestly

The value of a listing is its rating divided by its price.

Compute and show a dataframe with the 3 highest valued listings in each neighbourhood.

In [ ]:
# Still need to drop null values on prices and ratings
# I only managed to get the max higuest valued listing for each neighbourhood,
# still need to think how to get the top 3

# Madrid is pretty fucked up -> guess Stockholm has better data

basic_df_listings = df.select('name', 'city', 'neighbourhood', 'host_listings_count', extract_price_udf(f.col('price')).alias('price'), 'review_scores_rating')
basic_listings = basic_df_listings.filter(f.col('city') == 'Stockholm').select('neighbourhood', 'name', 'host_listings_count', f.round((f.col('price').cast("float")/f.col('review_scores_rating').cast("float"))).alias('valued_listings'))
higuest_listings = basic_listings.groupBy('neighbourhood').agg(f.max('valued_listings')).show(20)


## Trends
Now we want to analyze the "popularity" of your city. The data does not contain the number of bookings per listing, but we have a large number of reviews, and we will assume that this is a good indicator of activity on listings.

Compute and visualize the popularity (i.e., number of reviews) of your city over time.

In [23]:
# number of reviews of Madrid per month?
listings_reduced.filter(f.col('city') == 'Madrid').select(f.count('reviews_per_month').alias('total_reviews_per_month')).show(20)


+-----------------------+
|total_reviews_per_month|
+-----------------------+
|                  15767|
+-----------------------+



Compute and visualize the popularity of neighbourhoods over time. If there are many neighbourhoods in your city, you should select a few interesting ones for comparison.

In [35]:
listings_reduced.filter(f.col('city') == 'Madrid').groupBy('neighbourhood').agg(f.count('reviews_per_month').alias('total_reviews_per_month')).orderBy(f.col('total_reviews_per_month').desc()).show(20)


+------------------+-----------------------+
|     neighbourhood|total_reviews_per_month|
+------------------+-----------------------+
|            Centro|                   4356|
|       Embajadores|                   1018|
|               Sol|                    531|
|         Salamanca|                    509|
|          Malasaña|                    508|
|           Palacio|                    497|
|         La Latina|                    489|
|          Chamberí|                    473|
|        Arganzuela|                    462|
|          Justicia|                    459|
|            Aluche|                    452|
|       Carabanchel|                    447|
|Puente de Vallecas|                    424|
|     Ciudad Lineal|                    419|
|            Tetuán|                    302|
|            Retiro|                    291|
|            Cortes|                    284|
|           Moncloa|                    242|
|         Hortaleza|                    240|
|         

Compute and visualize the popularity of your city by season. For example, visualize the popularity of your city per month.

In [36]:
# number of reviews of Madrid per month?
listings_reduced.filter(f.col('city') == 'Madrid').select(f.count('reviews_per_month').alias('total_reviews_per_month')).show(20)

+-----------------------+
|total_reviews_per_month|
+-----------------------+
|                  15767|
+-----------------------+



## Reviews
In this part you should determine which words used in reviews that are the most positive. 

The individual reviews do not have a rating of the listing, so we will assume that each review gave the average rating to the listing, i.e., the one on the listing.

You should assign a positivity weight to each word seen in reviews and list the words with the highest weight. It is up to you to decide what the weight should be. For example, it can be a function of the rating on the listing on which it occurs, the number of reviews it occurs in, and the number of unique listings for which it was used to review.

Depending on your choice of weight function, you may also want to do some filtering of words. For example, remove words that only occur in a few reviews.

In [46]:
# Extract all comments from the reviews and append all of them in a dataframe
# Get a dictionary where we can have {word} : {times_seen} ??
# Look at the most frequent ones and remove the least frequent ones
# Asign weight to each of the words
# Weight: rating*n_reviews*n_listings
comment_dict = {}

def extract_comments(comments):
    for word in comments.split():
        if word not in comment_dict.keys():
            comment_dict[word] = 1  
        else:
            comment_dict[word] += 1
    print(comment_dict)


In [48]:
extract_comments_udf = f.udf(extract_comments)

In [ ]:
df_reviews_extracted = df_reviews.withColumn(extract_comments_udf(f.col('comments')))